In [1]:
from spike.integration.odin.annotator import OdinAnnotator
from spike.pattern_generation.utils import GenerationFromExamples, \
                                              PredictedRelationProvider
from spike.datamodel.dataset import FileBasedDataSet
from spike.pattern_generation.gen_pattern import PatternGenerator
from spike.pattern_generation.pattern_selectors import LabelEdgeSelector, TriggerVarNodeSelector

import numpy as np
import matplotlib.pyplot as plt

In [2]:
triggers = {'husband', 'wedding', 'marriage', 'divorced', 'marriages', 'bride', 'ex-wife', 'ex-husband', 'married', 'widow', 'widower', 'wife', 'marry', 'divorcing', 'sweetheart', 'divorce'}

with open('/Users/matane/matan/dev/datasets/generation_outputs/spouse/filtered_spouse_generation_outputs.txt', 'r') as f:
    text = f.read()

In [3]:
annotator = OdinAnnotator()
relation_id = "per:spouse"    

text = """Kate Middleton, the former British model who is married to Prince Harry of Wales, said of their first meeting : ''It was pretty surreal''\n\
It was Kate Middleton's sixth marriage to David, and she has four other children.\n\
As King prepared to leave for London on a three-day British holiday, aides suggested that he and his wife, Catherine, head to the beach near Point Mugu, the British Virgin Islands, for a one-on-one with King and a\n\
It was also confirmed that Prince William had been accompanied by a security detail for Prince Charles, who met his wife.\n\
She will also give birth to a son, Simon, from a previous marriage to actress Susan Sarandon.\n\
The couple met at Oxford University in 1957 and married less than two years later.\n\
In the movie, played for audience members and celebrities by James Brown, a divorced mother of two boys with a long career behind her ( he is an actor, not a director ), the two men share custody of their son Michael, 13, and\n\
The couple also have two sons, Sami and Jack, from previous marriages.\n\
In 2003, Kate Middleton announced that she was splitting from her husband of 35 years, Christopher and that he would step down from his leadership post as chief operating officer of the company.\n\
They have five children : Grace, from their marriage in 1996 ; Jameson, from their second marriage in 2002 ; Christopher, in 2004 ; Jake, from their third marriage and a daughter, Hannah Callaghan, from their fourth ; and two grandchildren."""

text = "Kate Middleton is walking down the street\n"+text

documents = []
for t in text.split('\n'):
    annotated = annotator.annotate_text(t)    
    documents.append(annotated)


In [4]:
always_process = True

def exactly_two_persons(sent):
    return [s.label for s in sent.spans['entities']].count('PERSON') == 2

docs_and_sents = []
for d in documents:
    for s in d.sentences:
        if always_process or exactly_two_persons(s):
            docs_and_sents.append((d, s))

In [5]:
pattern_generator = PatternGenerator([TriggerVarNodeSelector(triggers)], LabelEdgeSelector(), [])
pattern_to_examples = GenerationFromExamples.gen_pattern_to_examples_dict(docs_and_sents,
                                                   relation_id, triggers, 
                                                   pattern_generator,
                                                   relation_provider=PredictedRelationProvider())

> /Users/matane/matan/dev/spike/server/spike/pattern_generation/utils.py(138)gen_pattern_to_examples_dict()
-> for sample, rel in GenerationFromExamples.sentence_to_annotated_samples(
(Pdb) sent
Sentence(words=['Kate', 'Middleton', 'is', 'walking', 'down', 'the', 'street'], document_start_char_offsets=[0, 5, 15, 18, 26, 31, 35], document_end_char_offsets=[4, 14, 17, 25, 30, 34, 41], labels={'lemmas': ['Kate', 'Middleton', 'be', 'walk', 'down', 'the', 'street'], 'tags': ['NNP', 'NNP', 'VBZ', 'VBG', 'RP', 'DT', 'NN']}, spans={'entities': [LabeledSpan(first=0, last=1, label='PERSON')], 'chunks': [LabeledSpan(first=0, last=1, label='NP'), LabeledSpan(first=2, last=3, label='VP'), LabeledSpan(first=4, last=4, label='PRT'), LabeledSpan(first=5, last=6, label='NP')]}, graphs={'universal-enhanced': Graph(roots=[Span(first=3, last=3)], edges=[Edge(source=Span(first=1, last=1), target=Span(first=0, last=0), label='compound'), Edge(source=Span(first=3, last=3), target=Span(first=1, last=1), label

(Pdb) exit


BdbQuit: 

In [ ]:
# sorted_pattern_to_examples = sorted(pattern_to_examples.items(), key=lambda kv: len(kv[1]), reverse=True)
out_matching_docs, out_matching_sents, out_matching_sent_extractions, \
                          out_patterns, out_simple_patterns, out_counts = \
                          GenerationFromExamples.prepare_patterns_for_display(pattern_to_examples)

In [ ]:
fromm = 0
to = -1
plt.figure(figsize=(5, 5))
plt.bar(range(len(out_counts[fromm:to])), out_counts[fromm:to])
plt.title("Diversity")
plt.show()

In [20]:
idx = 0
print(out_patterns[idx])
print(out_simple_patterns[idx])

| trigger=[ word=/wedding|divorced|marry|married|wife|divorce|bride|sweetheart|widow|ex-wife|marriages|marriage|husband|widower|divorcing|ex-husband/ ]
  subject:Person=<nmod_from >conj_and
  object:Person=<nmod_from >conj_and
They have five children : Grace , from their [t marriage] in 1996 ; [s Jameson] , from their second marriage in 2002 ; [o Christopher] , in 2004 ; Jake , from their third marriage and a daughter , Hannah Callaghan , from their fourth ; and two grandchildren .


In [9]:
sum(out_counts)

96